In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import re # regular expression libary.
import nltk as nlp
from sklearn.feature_extraction.text import CountVectorizer 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
file_name="route"
df = pd.read_excel(file_name, sheet_name=None)
df_copy=df.copy()

In [ ]:
from nltk.corpus import stopwords
nlp.download("stopwords")

def clearText(textList):
    text_list=[]

    for text in textList:
        text = text.lower()  #Büyük harften -Küçük harfe çevirme
        text = re.sub("[^abcçdefgğhıijklmnoöprsştuüvyz]"," ",text)
        text=nlp.word_tokenize(text) # splits the words that are in the sentence from each other.
        text =[word for word in text if not word in set(stopwords.words("turkish"))]
        lemma=nlp.WordNetLemmatizer()
        text=[lemma.lemmatize(word) for word in text] # this code finds the root of the word for a word in the sentence and change them to their root form.
        text=" ".join(text)
        text_list.append(text) # store sentences in list

    return text_list

text_list=clearText(df_copy["YL"].text)
train_text_list=clearText(df_copy["Ek2"][:1400].text)
test_text_list=clearText(df_copy["Ek2"][1400:].text)

In [ ]:
train_y=LabelEncoder().fit_transform(df_copy["Ek2"][:1400].categoryText)
test_y=LabelEncoder().fit_transform(df_copy["Ek2"][1400:].categoryText)

In [ ]:
Train_X=train_text_list
Test_X=test_text_list
Test_Y=test_y
Train_Y=train_y

Tfidf_vect = TfidfVectorizer(max_features=500)
Tfidf_vect.fit(text_list)

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

In [ ]:
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# #Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

In [ ]:
naiveb=confusion_matrix(test_y,predictions_NB)
plt.figure(figsize=(10,10))
sns.heatmap(confusion_matrix(test_y,predictions_NB),annot=True,fmt='d')
plt.savefig('NaiveBayestek.eps', format='eps')

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(confusion_matrix(Test_Y,predictions_SVM),annot=True,fmt='d') 
plt.savefig('Svmtek.eps', format='eps')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
sentences_train=Train_X
sentences_test=Test_X
y_train=train_y
y_test=test_y
vectorizer = CountVectorizer()
vectorizer.fit(text_list)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
predictions_LR = classifier.predict(X_test)
print("Accuracy:", score)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(confusion_matrix(y_test,predictions_LR),annot=True,fmt='d')
plt.savefig('LogisticRegtek.eps', format='eps')